## **Extraer datos Generales sin filtros**

In [0]:
import requests
import pandas as pd
import os
import time  # Importamos time para pausar el script con sleep

# Leer el token desde el archivo CSV previamente guardado
df_TokenLargoPlazo = pd.read_csv("/dbfs/mnt/Token_LargoPlazo/long_lived_token.csv")
access_token = df_TokenLargoPlazo["access_token"].iloc[0]

# ID de la cuenta publicitaria de Meta
ad_account_id = "1952584805183463"

# Crear un directorio para guardar los datos si no existe
processed_dir = '/dbfs/mnt/processed/Ads_GralTotal'
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

# Función para obtener todas las campañas de una cuenta publicitaria con paginación
def obtener_campanasMeta(access_token, ad_account_id):
    url = f"https://graph.facebook.com/v13.0/act_{ad_account_id}/campaigns"
    params = {
        "access_token": access_token,
        "fields": "id,name,status",
        "limit": 25
    }
    all_campaigns = []
    while url:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            all_campaigns.extend(data['data'])
            url = data.get('paging', {}).get('next')
        else:
            print(f"Error al obtener campañas {response.status_code}: {response.text}")
            break
    return pd.DataFrame(all_campaigns) if all_campaigns else None

# Función para obtener conjuntos de anuncios de una campaña con paginación
def obtener_conjuntos_anunciosMeta(access_token, campaign_id):
    url = f"https://graph.facebook.com/v13.0/{campaign_id}/adsets"
    params = {
        "access_token": access_token,
        "fields": "id,name,status,effective_status",
        "limit": 25
    }
    all_adsets = []
    while url:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            all_adsets.extend(data['data'])
            url = data.get('paging', {}).get('next')
        else:
            print(f"Error al obtener conjuntos de anuncios {response.status_code}: {response.text}")
            break
    return pd.DataFrame(all_adsets) if all_adsets else None

# Función para obtener anuncios de un conjunto de anuncios con paginación
def obtener_anunciosMeta(access_token, adset_id):
    url = f"https://graph.facebook.com/v13.0/{adset_id}/ads"
    params = {
        "access_token": access_token,
        "fields": "id,name,effective_status",
        "limit": 25
    }
    all_ads = []
    while url:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            all_ads.extend(data['data'])
            url = data.get('paging', {}).get('next')
        else:
            print(f"Error al obtener anuncios {response.status_code}: {response.text}")
            break
    return pd.DataFrame(all_ads) if all_ads else None

# Función para obtener métricas (insights) de un anuncio
def obtener_alcance_anuncio(access_token, ad_id):
    url = f"https://graph.facebook.com/v13.0/{ad_id}/insights"
    params = {
        "access_token": access_token,
        "fields": "ad_id,ad_name,adset_name,campaign_name,date_start,date_stop,reach,impressions,frequency,spend,clicks,cost_per_ad_click,inline_link_clicks,conversion_rate_ranking,cpc,cpp,cpm,actions,ad_click_actions,quality_ranking,conversions,buying_type",
        "level": "ad"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()['data']
        return pd.DataFrame(data) if data else None
    else:
        print(f"Error al obtener insights del anuncio {response.status_code}: {response.text}")
        return None

# Flujo principal
df_campanas = obtener_campanasMeta(access_token, ad_account_id)
if df_campanas is not None:
    all_insights = []  # Lista para acumular los insights de todos los anuncios
    for i, campana in df_campanas.iterrows():
        print(f"Procesando campaña {i + 1} de {len(df_campanas)}: {campana['name']}")
        campaign_id = campana['id']
        df_conjuntos = obtener_conjuntos_anunciosMeta(access_token, campaign_id)
        
        if df_conjuntos is not None:
            for j, conjunto in df_conjuntos.iterrows():
                print(f"  Procesando conjunto de anuncios {j + 1} de {len(df_conjuntos)}: {conjunto['name']}")
                adset_id = conjunto['id']
                df_anuncios = obtener_anunciosMeta(access_token, adset_id)
                
                if df_anuncios is not None:
                    for k, anuncio in df_anuncios.iterrows():
                        print(f"    Procesando anuncio {k + 1} de {len(df_anuncios)}: {anuncio['name']}")
                        ad_id = anuncio['id']
                        df_anuncio_insights = obtener_alcance_anuncio(access_token, ad_id)
                        if df_anuncio_insights is not None:
                            all_insights.append(df_anuncio_insights)

        # Guardar insights al finalizar una campaña
        if all_insights:
            df_all_insights = pd.concat(all_insights, ignore_index=True)
            df_all_insights.to_csv(f"{processed_dir}/ads_GralTotal.csv", index=False)
            print(f"Campaña guardada en el archivo: {campana['name']}")

else:
    print("No se encontraron campañas en la cuenta publicitaria.")

# Verificar archivos creados
dbutils.fs.ls("/mnt/processed/Ads_GralTotal")


Procesando campaña 1 de 53: Test_All_Nov_Camp
  Procesando conjunto de anuncios 1 de 14: Test_Group_FerCreditEncinos
    Procesando anuncio 1 de 4: Test_Ad_FerCredit_BanorteSab
    Procesando anuncio 2 de 4: Test_Ad_FerCredit_BBVAViernes
    Procesando anuncio 3 de 4: Test_Ad_FerCreditGral
    Procesando anuncio 4 de 4: Test_Ad_FerCredit_Domingo
  Procesando conjunto de anuncios 2 de 14: Test_Group_Ads_Generales
    Procesando anuncio 1 de 1: Test_Ad_BosqueDesertAlta
  Procesando conjunto de anuncios 3 de 14: Test_Group_MemesTrafico
    Procesando anuncio 1 de 4: Test_Ad_MemTraffic146Encinos
    Procesando anuncio 2 de 4: Test_Ad_MemTraffic106Alta
    Procesando anuncio 3 de 4: Test_Ad_MemTrafficT80Altavista
    Procesando anuncio 4 de 4: Test_Ad_MemTraffic106Encinos
  Procesando conjunto de anuncios 4 de 14: Test_Group_VideosGeneralesPt2
    Procesando anuncio 1 de 5: Test_AdVideo_LosArtistas
    Procesando anuncio 2 de 5: Test_AdVideo_PreNavidad
    Procesando anuncio 3 de 5: Test_Ad

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:730)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:448)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:448)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

- probando paginacion(limite que se cree 25 campañas) FUNCIONA 

In [0]:
import requests
import pandas as pd
import os
import time  # Importamos time para pausar el script con sleep

# Leer el token desde el archivo CSV previamente guardado
df_TokenLargoPlazo = pd.read_csv("/dbfs/mnt/Token_LargoPlazo/long_lived_token.csv")
access_token = df_TokenLargoPlazo["access_token"].iloc[0]

# ID de la cuenta publicitaria de Meta
ad_account_id = "1952584805183463"

# Crear un directorio para guardar los datos si no existe
processed_dir = '/dbfs/mnt/processed/Ads_GralTotal'
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

# Función para obtener todas las campañas de una cuenta publicitaria con paginación
def obtener_campanasMeta(access_token, ad_account_id):
    """
    Obtiene todas las campañas asociadas a una cuenta publicitaria, usando paginación para manejar resultados grandes.
    """
    url = f"https://graph.facebook.com/v13.0/act_{ad_account_id}/campaigns"
    params = {
        "access_token": access_token,
        "fields": "id,name,status",  # Campos específicos a recuperar
        "limit": 25  # Límite por página
    }
    all_campaigns = []  # Lista para acumular todas las campañas
    while url:  # Mientras exista una URL para la siguiente página
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            all_campaigns.extend(data['data'])
            url = data.get('paging', {}).get('next')  # Obtener la siguiente URL
        else:
            print(f"Error al obtener campañas {response.status_code}: {response.text}")
            break
    return pd.DataFrame(all_campaigns) if all_campaigns else None

# Función para obtener conjuntos de anuncios de una campaña con paginación
def obtener_conjuntos_anunciosMeta(access_token, campaign_id):
    """
    Obtiene todos los conjuntos de anuncios para una campaña específica, usando paginación.
    """
    url = f"https://graph.facebook.com/v13.0/{campaign_id}/adsets"
    params = {
        "access_token": access_token,
        "fields": "id,name,status,effective_status",
        "limit": 25  # Límite por página
    }
    all_adsets = []  # Lista para acumular todos los conjuntos de anuncios
    while url:  # Mientras exista una URL para la siguiente página
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            all_adsets.extend(data['data'])
            url = data.get('paging', {}).get('next')  # Obtener la siguiente URL
        else:
            print(f"Error al obtener conjuntos de anuncios {response.status_code}: {response.text}")
            break
    return pd.DataFrame(all_adsets) if all_adsets else None

# Función para obtener anuncios de un conjunto de anuncios con paginación
def obtener_anunciosMeta(access_token, adset_id):
    """
    Obtiene todos los anuncios asociados a un conjunto de anuncios, usando paginación.
    """
    url = f"https://graph.facebook.com/v13.0/{adset_id}/ads"
    params = {
        "access_token": access_token,
        "fields": "id,name,effective_status",
        "limit": 25  # Límite por página
    }
    all_ads = []  # Lista para acumular todos los anuncios
    while url:  # Mientras exista una URL para la siguiente página
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            all_ads.extend(data['data'])
            url = data.get('paging', {}).get('next')  # Obtener la siguiente URL
        else:
            print(f"Error al obtener anuncios {response.status_code}: {response.text}")
            break
    return pd.DataFrame(all_ads) if all_ads else None

# Función para obtener métricas (insights) de un anuncio
def obtener_alcance_anuncio(access_token, ad_id):
    """
    Recupera las métricas (insights) de un anuncio específico.
    """
    url = f"https://graph.facebook.com/v13.0/{ad_id}/insights"
    params = {
        "access_token": access_token,
        "fields": "ad_id,ad_name,adset_name,campaign_name,date_start,date_stop,reach,impressions,frequency,spend,clicks,cost_per_ad_click,inline_link_clicks,conversion_rate_ranking,cpc,cpp,cpm,actions,ad_click_actions,quality_ranking,conversions,buying_type",  # Métricas solicitadas
        "level": "ad"  # Nivel de la métrica (anuncio)
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()['data']
        return pd.DataFrame(data) if data else None
    else:
        print(f"Error al obtener insights del anuncio {response.status_code}: {response.text}")
        return None

# Flujo principal
df_campanas = obtener_campanasMeta(access_token, ad_account_id)  # Obtener todas las campañas
if df_campanas is not None:
    all_insights = []  # Lista para acumular los insights de todos los anuncios
    for i, campana in df_campanas.iterrows():
        print(f"Procesando campaña {i + 1} de {len(df_campanas)}: {campana['name']}")
        campaign_id = campana['id']
        df_conjuntos = obtener_conjuntos_anunciosMeta(access_token, campaign_id)
        
        # Verificar si excede el límite de conjuntos de anuncios
        if df_conjuntos is not None:
            if len(df_conjuntos) > 13:
                print("Se alcanzaron más de 14 conjuntos de anuncios. Pausando 5 minutos...")
                time.sleep(300)

            for j, conjunto in df_conjuntos.iterrows():
                print(f"  Procesando conjunto de anuncios {j + 1} de {len(df_conjuntos)}: {conjunto['name']}")
                adset_id = conjunto['id']
                df_anuncios = obtener_anunciosMeta(access_token, adset_id)
                
                # Verificar si excede el límite de anuncios
                if df_anuncios is not None:
                    if len(df_anuncios) > 33:
                        print("Se alcanzaron más de 34 anuncios. Pausando 5 minutos...")
                        time.sleep(300)

                    for k, anuncio in df_anuncios.iterrows():
                        print(f"    Procesando anuncio {k + 1} de {len(df_anuncios)}: {anuncio['name']}")
                        ad_id = anuncio['id']
                        df_anuncio_insights = obtener_alcance_anuncio(access_token, ad_id)
                        if df_anuncio_insights is not None:
                            all_insights.append(df_anuncio_insights)

        # Pausa adicional por cada campaña
        print("Esperando 5 minutos antes de procesar la siguiente campaña...")
        time.sleep(300)

    # Guardar todos los insights en un archivo CSV
    if all_insights:
        df_all_insights = pd.concat(all_insights, ignore_index=True)
        df_all_insights.to_csv(f"{processed_dir}/ads_GralTotal.csv", index=False)
        print("Todos los insights de los anuncios guardados exitosamente.")
    else:
        print("No se encontraron insights para los anuncios.")
else:
    print("No se encontraron campañas en la cuenta publicitaria.")

# Verificar archivos creados
dbutils.fs.ls("/mnt/processed/Ads_GralTotal")


Procesando campaña 1 de 53: Test_All_Nov_Camp
Se alcanzaron más de 14 conjuntos de anuncios. Pausando 5 minutos...
  Procesando conjunto de anuncios 1 de 14: Test_Group_FerCreditEncinos
    Procesando anuncio 1 de 4: Test_Ad_FerCredit_BanorteSab
    Procesando anuncio 2 de 4: Test_Ad_FerCredit_BBVAViernes
    Procesando anuncio 3 de 4: Test_Ad_FerCreditGral
    Procesando anuncio 4 de 4: Test_Ad_FerCredit_Domingo
  Procesando conjunto de anuncios 2 de 14: Test_Group_Ads_Generales
    Procesando anuncio 1 de 1: Test_Ad_BosqueDesertAlta
  Procesando conjunto de anuncios 3 de 14: Test_Group_MemesTrafico
    Procesando anuncio 1 de 4: Test_Ad_MemTraffic146Encinos
    Procesando anuncio 2 de 4: Test_Ad_MemTraffic106Alta
    Procesando anuncio 3 de 4: Test_Ad_MemTrafficT80Altavista
    Procesando anuncio 4 de 4: Test_Ad_MemTraffic106Encinos
  Procesando conjunto de anuncios 4 de 14: Test_Group_VideosGeneralesPt2
    Procesando anuncio 1 de 5: Test_AdVideo_LosArtistas
    Procesando anuncio 2

[FileInfo(path='dbfs:/mnt/processed/Ads_GralTotal/ads_GralTotal.csv', name='ads_GralTotal.csv', size=64019, modificationTime=1731614069000)]

sin paginacion, funcional pero solo obtiene 25 campañas, creo que es el limite de pagina

In [0]:
import requests
import pandas as pd
import os
import time  # Importamos time para pausar el script con sleep

# Leer el token desde el archivo CSV previamente guardado
df_TokenLargoPlazo = pd.read_csv("/dbfs/mnt/Token_LargoPlazo/long_lived_token.csv")
access_token = df_TokenLargoPlazo["access_token"].iloc[0]

# ID de la cuenta publicitaria de Meta
ad_account_id = "1952584805183463"

# Crear un directorio para guardar los datos si no existe
processed_dir = '/dbfs/mnt/processed/Ads_GralTotal'
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

# Función para obtener todas las campañas de una cuenta publicitaria
def obtener_campanasMeta(access_token, ad_account_id):
    """
    Obtiene todas las campañas asociadas a una cuenta publicitaria.
    """
    url = f"https://graph.facebook.com/v13.0/act_{ad_account_id}/campaigns"
    params = {
        "access_token": access_token,
        "fields": "id,name,status"  # Campos específicos a recuperar
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()['data']
        return pd.DataFrame(data) if data else None
    else:
        print(f"Error al obtener campañas {response.status_code}: {response.text}")
        return None

# Función para obtener conjuntos de anuncios de una campaña
def obtener_conjuntos_anunciosMeta(access_token, campaign_id):
    """
    Obtiene todos los conjuntos de anuncios para una campaña específica.
    """
    url = f"https://graph.facebook.com/v13.0/{campaign_id}/adsets"
    params = {
        "access_token": access_token,
        "fields": "id,name,status,effective_status"  # Campos requeridos
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()['data']
        return pd.DataFrame(data) if data else None
    else:
        print(f"Error al obtener conjuntos de anuncios {response.status_code}: {response.text}")
        return None

# Función para obtener anuncios de un conjunto de anuncios
def obtener_anunciosMeta(access_token, adset_id):
    """
    Obtiene todos los anuncios asociados a un conjunto de anuncios.
    """
    url = f"https://graph.facebook.com/v13.0/{adset_id}/ads"
    params = {
        "access_token": access_token,
        "fields": "id,name,effective_status"  # Campos requeridos
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()['data']
        return pd.DataFrame(data) if data else None
    else:
        print(f"Error al obtener anuncios {response.status_code}: {response.text}")
        return None

# Función para obtener métricas (insights) de un anuncio
def obtener_alcance_anuncio(access_token, ad_id):
    """
    Recupera las métricas (insights) de un anuncio específico.
    """
    url = f"https://graph.facebook.com/v13.0/{ad_id}/insights"
    params = {
        "access_token": access_token,
        "fields": "ad_id,ad_name,adset_name,campaign_name,date_start,date_stop,reach,impressions,frequency,spend,clicks,cost_per_ad_click,inline_link_clicks,conversion_rate_ranking,cpc,cpp,cpm,actions,ad_click_actions,quality_ranking,conversions,buying_type",  # Métricas solicitadas
        "level": "ad"  # Nivel de la métrica (anuncio)
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()['data']
        return pd.DataFrame(data) if data else None
    else:
        print(f"Error al obtener insights del anuncio {response.status_code}: {response.text}")
        return None

# Flujo principal
df_campanas = obtener_campanasMeta(access_token, ad_account_id)  # Obtener todas las campañas
if df_campanas is not None:
    all_insights = []  # Lista para acumular los insights de todos los anuncios
    for i, campana in df_campanas.iterrows():
        print(f"Procesando campaña {i + 1} de {len(df_campanas)}: {campana['name']}")
        campaign_id = campana['id']
        df_conjuntos = obtener_conjuntos_anunciosMeta(access_token, campaign_id)
        
        # Verificar si excede el límite de conjuntos de anuncios
        if df_conjuntos is not None:
            if len(df_conjuntos) > 13:
                print("Se alcanzaron más de 14 conjuntos de anuncios. Pausando 5 minutos...")
                time.sleep(300)

            for j, conjunto in df_conjuntos.iterrows():
                print(f"  Procesando conjunto de anuncios {j + 1} de {len(df_conjuntos)}: {conjunto['name']}")
                adset_id = conjunto['id']
                df_anuncios = obtener_anunciosMeta(access_token, adset_id)
                
                # Verificar si excede el límite de anuncios
                if df_anuncios is not None:
                    if len(df_anuncios) > 33:
                        print("Se alcanzaron más de 34 anuncios. Pausando 5 minutos...")
                        time.sleep(300)

                    for k, anuncio in df_anuncios.iterrows():
                        print(f"    Procesando anuncio {k + 1} de {len(df_anuncios)}: {anuncio['name']}")
                        ad_id = anuncio['id']
                        df_anuncio_insights = obtener_alcance_anuncio(access_token, ad_id)
                        if df_anuncio_insights is not None:
                            all_insights.append(df_anuncio_insights)
        # Pausa adicional por cada campaña
        print("Esperando 5 minutos antes de procesar la siguiente campaña...")
        time.sleep(300)

    # Guardar todos los insights en un archivo CSV
    if all_insights:
        df_all_insights = pd.concat(all_insights, ignore_index=True)
        df_all_insights.to_csv(f"{processed_dir}/ads_GralTotalPrueba14112024.csv", index=False)
        print("Todos los insights de los anuncios guardados exitosamente.")
    else:
        print("No se encontraron insights para los anuncios.")
else:
    print("No se encontraron campañas en la cuenta publicitaria.")

# Verificar archivos creados
dbutils.fs.ls("/mnt/processed/Ads_GralTotal")


Procesando campaña 1 de 25: Test_All_Nov_Camp
Se alcanzaron más de 14 conjuntos de anuncios. Pausando 5 minutos...
  Procesando conjunto de anuncios 1 de 14: Test_Group_FerCreditEncinos
    Procesando anuncio 1 de 4: Test_Ad_FerCredit_BanorteSab
    Procesando anuncio 2 de 4: Test_Ad_FerCredit_BBVAViernes
    Procesando anuncio 3 de 4: Test_Ad_FerCreditGral
    Procesando anuncio 4 de 4: Test_Ad_FerCredit_Domingo
  Procesando conjunto de anuncios 2 de 14: Test_Group_Ads_Generales
    Procesando anuncio 1 de 1: Test_Ad_BosqueDesertAlta
  Procesando conjunto de anuncios 3 de 14: Test_Group_MemesTrafico
    Procesando anuncio 1 de 4: Test_Ad_MemTraffic146Encinos
    Procesando anuncio 2 de 4: Test_Ad_MemTraffic106Alta
    Procesando anuncio 3 de 4: Test_Ad_MemTrafficT80Altavista
    Procesando anuncio 4 de 4: Test_Ad_MemTraffic106Encinos
  Procesando conjunto de anuncios 4 de 14: Test_Group_VideosGeneralesPt2
    Procesando anuncio 1 de 5: Test_AdVideo_LosArtistas
    Procesando anuncio 2

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:730)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:448)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:448)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

prueba funcional (genera 25 campañas , esta por analizar si genera mas y porque)

In [0]:
import requests
import pandas as pd
import os
import time  # Importar time para usar sleep

# Leer el token desde el archivo CSV previamente guardado
df_TokenLargoPlazo = pd.read_csv("/dbfs/mnt/Token_LargoPlazo/long_lived_token.csv")
access_token = df_TokenLargoPlazo["access_token"].iloc[0]

# Reemplaza este valor con tu ad_account_id (ID de la cuenta publicitaria de Meta)
ad_account_id = "1952584805183463"

# Crear el directorio '/dbfs:/mnt/processed' si no existe
processed_dir = '/dbfs/mnt/processed/Ads_GralTotal'
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

# Función para obtener todas las campañas de una cuenta publicitaria
def obtener_campanasMeta(access_token, ad_account_id):
    url = f"https://graph.facebook.com/v13.0/act_{ad_account_id}/campaigns"
    params = {
        "access_token": access_token,
        "fields": "id,name,status"  # Solo los campos necesarios
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()['data']
        return pd.DataFrame(data) if data else None
    else:
        print(f"Error al obtener campañas {response.status_code}: {response.text}")
        return None

# Función para obtener todos los conjuntos de anuncios de todas las campañas
def obtener_conjuntos_anunciosMeta(access_token, campaign_id):
    url = f"https://graph.facebook.com/v13.0/{campaign_id}/adsets"
    params = {
        "access_token": access_token,
        "fields": "id,name,status,effective_status"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()['data']
        return pd.DataFrame(data) if data else None
    else:
        print(f"Error al obtener conjuntos de anuncios {response.status_code}: {response.text}")
        return None

# Función para obtener los anuncios de un conjunto de anuncios
def obtener_anunciosMeta(access_token, adset_id):
    url = f"https://graph.facebook.com/v13.0/{adset_id}/ads"
    params = {
        "access_token": access_token,
        "fields": "id,name,effective_status"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()['data']
        return pd.DataFrame(data) if data else None
    else:
        print(f"Error al obtener anuncios {response.status_code}: {response.text}")
        return None

# Función para obtener el alcance de un anuncio mediante el endpoint de insights
def obtener_alcance_anuncio(access_token, ad_id):
    url = f"https://graph.facebook.com/v13.0/{ad_id}/insights"
    params = {
        "access_token": access_token,
        "fields": "ad_id,ad_name,adset_name,campaign_name,date_start,date_stop,reach,impressions,frequency,spend,clicks,cost_per_ad_click,inline_link_clicks,conversion_rate_ranking,cpc,cpp,cpm,actions,ad_click_actions,quality_ranking,conversions,buying_type",
        "level": "ad"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()['data']
        return pd.DataFrame(data) if data else None
    else:
        print(f"Error al obtener insights del anuncio {response.status_code}: {response.text}")
        return None

# Flujo principal
df_campanas = obtener_campanasMeta(access_token, ad_account_id)
if df_campanas is not None:
    all_insights = []
    for i, campana in df_campanas.iterrows():
        print(f"Procesando campaña {i + 1} de {len(df_campanas)}")
        campaign_id = campana['id']
        df_conjuntos = obtener_conjuntos_anunciosMeta(access_token, campaign_id)
        if df_conjuntos is not None:
            for j, conjunto in df_conjuntos.iterrows():
                print(f"  Procesando conjunto de anuncios {j + 1} de {len(df_conjuntos)}")
                adset_id = conjunto['id']
                df_anuncios = obtener_anunciosMeta(access_token, adset_id)
                if df_anuncios is not None:
                    for k, anuncio in df_anuncios.iterrows():
                        print(f"    Procesando anuncio {k + 1} de {len(df_anuncios)}")
                        ad_id = anuncio['id']
                        df_anuncio_insights = obtener_alcance_anuncio(access_token, ad_id)
                        if df_anuncio_insights is not None:
                            all_insights.append(df_anuncio_insights)
        # Pausa de 5 minutos entre campañas
        print("Esperando 5 minutos antes de continuar con la siguiente campaña...")
        time.sleep(300)

    # Guardar todos los insights en un solo archivo CSV
    if all_insights:
        df_all_insights = pd.concat(all_insights, ignore_index=True)
        df_all_insights.to_csv(f"{processed_dir}/ads_GralTotal.csv", index=False)
        print("Todos los insights de los anuncios guardados exitosamente.")
    else:
        print("No se encontraron insights para los anuncios.")
else:
    print("No se encontraron campañas en la cuenta publicitaria.")

# Verificar archivos creados
dbutils.fs.ls("/mnt/processed/Ads_GralTotal")





Procesando campaña 1 de 25
  Procesando conjunto de anuncios 1 de 14
    Procesando anuncio 1 de 4
    Procesando anuncio 2 de 4
    Procesando anuncio 3 de 4
    Procesando anuncio 4 de 4
  Procesando conjunto de anuncios 2 de 14
    Procesando anuncio 1 de 1
  Procesando conjunto de anuncios 3 de 14
    Procesando anuncio 1 de 4
    Procesando anuncio 2 de 4
    Procesando anuncio 3 de 4
    Procesando anuncio 4 de 4
  Procesando conjunto de anuncios 4 de 14
    Procesando anuncio 1 de 5
    Procesando anuncio 2 de 5
    Procesando anuncio 3 de 5
    Procesando anuncio 4 de 5
    Procesando anuncio 5 de 5
  Procesando conjunto de anuncios 5 de 14
    Procesando anuncio 1 de 3
    Procesando anuncio 2 de 3
    Procesando anuncio 3 de 3
  Procesando conjunto de anuncios 6 de 14
    Procesando anuncio 1 de 2
    Procesando anuncio 2 de 2
  Procesando conjunto de anuncios 7 de 14
    Procesando anuncio 1 de 4
    Procesando anuncio 2 de 4
    Procesando anuncio 3 de 4
    Procesando anun

[FileInfo(path='dbfs:/mnt/processed/Ads_GralTotal/ads_GralTotal.csv', name='ads_GralTotal.csv', size=70417, modificationTime=1731535700000)]

In [0]:
#codigo para guardar todos los insights necesarios para el analisis de datos de SOLO los anuncios
import requests
import pandas as pd
import os

# Leer el token desde el archivo CSV previamente guardado
df_TokenLargoPlazo = pd.read_csv("/dbfs/mnt/Token_LargoPlazo/long_lived_token.csv")
access_token = df_TokenLargoPlazo["access_token"].iloc[0]

# Reemplaza este valor con tu ad_account_id (ID de la cuenta publicitaria de Meta)
ad_account_id = "1952584805183463"

# Crear el directorio '/dbfs:/mnt/processed' si no existe
processed_dir = '/dbfs/mnt/processed/Ads_GralTotal'
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

# Función para obtener todos los conjuntos de anuncios de todas las campañas
def obtener_conjuntos_anunciosMeta(access_token, ad_account_id):
    url = f"https://graph.facebook.com/v13.0/act_{ad_account_id}/adsets"
    params = {
        "access_token": access_token,
        "fields": "id,name,status"  # Especifica las columnas que deseas obtener
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()['data']
        df_conjuntosMeta = pd.DataFrame(data)
        return df_conjuntosMeta
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# Función para obtener el alcance de un anuncio mediante el endpoint de insights
def obtener_alcance_anuncio(access_token, ad_id):
    url = f"https://graph.facebook.com/v13.0/{ad_id}/insights"
    params = {
        "access_token": access_token,
        "fields": "ad_id,ad_name,adset_name,campaign_name,date_start,date_stop,reach,impressions,frequency,spend,clicks,cost_per_ad_click,inline_link_clicks,conversion_rate_ranking,cpc,cpp,cpm,actions,ad_click_actions,quality_ranking,conversions,buying_type",  # Especifica las métricas que deseas obtener
        "level": "ad",  # Nivel de la métrica, en este caso es a nivel de anuncio
        
    }

    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()['data']
        df_insights = pd.DataFrame(data)
        return df_insights
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# Obtener todos los conjuntos de anuncios
df_conjuntos = obtener_conjuntos_anunciosMeta(access_token, ad_account_id)

if df_conjuntos is not None:
    all_insights = []
    # Obtener y guardar el alcance y otras métricas para cada anuncio en los conjuntos de anuncios
    for adset_id in df_conjuntos['id']:
        url = f"https://graph.facebook.com/v13.0/{adset_id}/ads"
        params = {
            "access_token": access_token,
            "fields": "id,name"  # Obtener solo los IDs de los anuncios
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            ads_data = response.json()['data']
            for ad in ads_data:
                ad_id = ad['id']
                df_anuncio_insights = obtener_alcance_anuncio(access_token, ad_id)
                if df_anuncio_insights is not None:
                    all_insights.append(df_anuncio_insights)

    # Unir todos los DataFrames de insights en uno solo
    if all_insights:
        df_all_insights = pd.concat(all_insights, ignore_index=True)
        # Guardar todos los insights en un solo CSV
        df_all_insights.to_csv(f"{processed_dir}/ads_GralTotal.csv", index=False)
        print("Todos los insights de los anuncios generales por hora guardados exitosamente.")
    else:
        print("No se encontraron insights de anuncios.")
else:
    print("No se encontraron conjuntos de anuncios o limite de solicitud excedido.")
#revisar cuales archivos existen dentro de la ruta
dbutils.fs.ls("/mnt/processed/Ads_GralTotal")

Todos los insights de los anuncios generales por hora guardados exitosamente.


[FileInfo(path='dbfs:/mnt/processed/Ads_GralTotal/ads_GralTotal.csv', name='ads_GralTotal.csv', size=26528, modificationTime=1731522407000)]

In [0]:
#revisar cuales archivos existen dentro de la ruta
dbutils.fs.ls("/mnt/processed/Ads_GralTotal")

[FileInfo(path='dbfs:/mnt/processed/Ads_GralTotal/ads_GralTotal.csv', name='ads_GralTotal.csv', size=70417, modificationTime=1731535700000)]

In [0]:
import pandas as pd

# Cargar el archivo generado
file_path = '/dbfs/mnt/processed/Ads_GralTotal/ads_GralTotal.csv'
df = pd.read_csv(file_path)

# Ver las dimensiones del DataFrame
print(f"Total filas: {df.shape[0]}, Total columnas: {df.shape[1]}")

# Mostrar una muestra de los datos
display(df.head(200))  # Cambia el número para mostrar más filas

Total filas: 75, Total columnas: 19


ad_id,ad_name,adset_name,campaign_name,date_start,date_stop,reach,impressions,frequency,spend,clicks,inline_link_clicks,conversion_rate_ranking,cpc,cpp,cpm,actions,quality_ranking,buying_type
120211392194460669,Test_Ad_FerCredit_BanorteSab,Test_Group_FerCreditEncinos,Test_All_Nov_Camp,2024-10-15,2024-11-13,1661.0,2521.0,1.51776,135.05,117.0,44.0,UNKNOWN,1.154274,81.306442,53.570012,"[{'action_type': 'onsite_conversion.total_messaging_connection', 'value': '3'}, {'action_type': 'onsite_conversion.messaging_first_reply', 'value': '3'}, {'action_type': 'post_engagement', 'value': '45'}, {'action_type': 'page_engagement', 'value': '45'}, {'action_type': 'onsite_conversion.messaging_user_depth_3_message_send', 'value': '2'}, {'action_type': 'onsite_conversion.messaging_user_depth_2_message_send', 'value': '2'}, {'action_type': 'onsite_conversion.messaging_conversation_started_7d', 'value': '3'}, {'action_type': 'post_reaction', 'value': '1'}, {'action_type': 'link_click', 'value': '44'}]",UNKNOWN,AUCTION
120211391623690669,Test_Ad_FerCredit_BBVAViernes,Test_Group_FerCreditEncinos,Test_All_Nov_Camp,2024-10-15,2024-11-13,85.0,119.0,1.4,7.77,5.0,1.0,UNKNOWN,1.554,91.411765,65.294118,"[{'action_type': 'post_engagement', 'value': '1'}, {'action_type': 'page_engagement', 'value': '1'}, {'action_type': 'link_click', 'value': '1'}]",UNKNOWN,AUCTION
120211391375820669,Test_Ad_FerCreditGral,Test_Group_FerCreditEncinos,Test_All_Nov_Camp,2024-10-15,2024-11-13,1756.0,2734.0,1.556948,145.37,156.0,26.0,UNKNOWN,0.931859,82.784738,53.171178,"[{'action_type': 'onsite_conversion.total_messaging_connection', 'value': '2'}, {'action_type': 'post_engagement', 'value': '27'}, {'action_type': 'page_engagement', 'value': '27'}, {'action_type': 'onsite_conversion.messaging_conversation_started_7d', 'value': '1'}, {'action_type': 'post_reaction', 'value': '1'}, {'action_type': 'link_click', 'value': '26'}]",UNKNOWN,AUCTION
120211392468530669,Test_Ad_FerCredit_Domingo,Test_Group_FerCreditEncinos,Test_All_Nov_Camp,2024-10-15,2024-11-13,42.0,55.0,1.309524,4.02,2.0,null,UNKNOWN,2.01,95.714286,73.090909,null,UNKNOWN,AUCTION
120211362612890669,Test_Ad_BosqueDesertAlta,Test_Group_Ads_Generales,Test_All_Nov_Camp,2024-10-15,2024-11-13,4628.0,8159.0,1.762965,284.2,153.0,40.0,UNKNOWN,1.857516,61.408816,34.8327,"[{'action_type': 'onsite_conversion.total_messaging_connection', 'value': '1'}, {'action_type': 'post_engagement', 'value': '48'}, {'action_type': 'page_engagement', 'value': '49'}, {'action_type': 'onsite_conversion.messaging_user_depth_3_message_send', 'value': '1'}, {'action_type': 'post', 'value': '1'}, {'action_type': 'onsite_conversion.messaging_user_depth_2_message_send', 'value': '1'}, {'action_type': 'like', 'value': '1'}, {'action_type': 'onsite_conversion.messaging_conversation_started_7d', 'value': '1'}, {'action_type': 'post_reaction', 'value': '7'}, {'action_type': 'link_click', 'value': '40'}]",UNKNOWN,AUCTION
120211361743520669,Test_Ad_MemTraffic146Encinos,Test_Group_MemesTrafico,Test_All_Nov_Camp,2024-10-15,2024-11-13,1212.0,1732.0,1.429043,117.23,12.0,8.0,UNKNOWN,9.769167,96.724422,67.684758,"[{'action_type': 'post_engagement', 'value': '10'}, {'action_type': 'onsite_conversion.post_save', 'value': '1'}, {'action_type': 'page_engagement', 'value': '10'}, {'action_type': 'post_reaction', 'value': '1'}, {'action_type': 'link_click', 'value': '8'}]",UNKNOWN,AUCTION
120211362084550669,Test_Ad_MemTraffic106Alta,Test_Group_MemesTrafico,Test_All_Nov_Camp,2024-10-15,2024-11-13,1609.0,2585.0,1.606588,183.03,16.0,6.0,UNKNOWN,11.439375,113.753884,70.804642,"[{'action_type': 'post_engagement', 'value': '8'}, {'action_type': 'page_engagement', 'value': '8'}, {'action_type': 'post_reaction', 'value': '2'}, {'action_type': 'link_click', 'value': '6'}]",UNKNOWN,AUCTION
120211361894020669,Test_Ad_MemTrafficT80Altavista,Test_Group_MemesTrafico,Test_All_Nov_Camp,2024-10-15,2024-11-13,2746.0,4407.0,1.60488,306.86,54.0,25.0,UNKNOWN,5.682593,111.747997,69.630134,"[{

In [0]:
#revisar el contenido de cada columna
import pandas as pd
#crear dataframe para leer el csv
df = pd.read_csv("/dbfs/mnt/processed/Ads_GralTotal/ads_GralTotal.csv")

# Ver las columnas del DataFrame
display(df.columns)
display(df)

Index(['ad_id', 'ad_name', 'adset_name', 'campaign_name', 'date_start',
       'date_stop', 'reach', 'impressions', 'frequency', 'spend', 'clicks',
       'inline_link_clicks', 'conversion_rate_ranking', 'cpc', 'cpp', 'cpm',
       'actions', 'quality_ranking', 'buying_type'],
      dtype='object')

ad_id,ad_name,adset_name,campaign_name,date_start,date_stop,reach,impressions,frequency,spend,clicks,inline_link_clicks,conversion_rate_ranking,cpc,cpp,cpm,actions,quality_ranking,buying_type
120211392194460669,Test_Ad_FerCredit_BanorteSab,Test_Group_FerCreditEncinos,Test_All_Nov_Camp,2024-10-15,2024-11-13,1661.0,2521.0,1.51776,135.05,117.0,44.0,UNKNOWN,1.154274,81.306442,53.570012,"[{'action_type': 'onsite_conversion.total_messaging_connection', 'value': '3'}, {'action_type': 'onsite_conversion.messaging_first_reply', 'value': '3'}, {'action_type': 'post_engagement', 'value': '45'}, {'action_type': 'page_engagement', 'value': '45'}, {'action_type': 'onsite_conversion.messaging_user_depth_3_message_send', 'value': '2'}, {'action_type': 'onsite_conversion.messaging_user_depth_2_message_send', 'value': '2'}, {'action_type': 'onsite_conversion.messaging_conversation_started_7d', 'value': '3'}, {'action_type': 'post_reaction', 'value': '1'}, {'action_type': 'link_click', 'value': '44'}]",UNKNOWN,AUCTION
120211391623690669,Test_Ad_FerCredit_BBVAViernes,Test_Group_FerCreditEncinos,Test_All_Nov_Camp,2024-10-15,2024-11-13,85.0,119.0,1.4,7.77,5.0,1.0,UNKNOWN,1.554,91.411765,65.294118,"[{'action_type': 'post_engagement', 'value': '1'}, {'action_type': 'page_engagement', 'value': '1'}, {'action_type': 'link_click', 'value': '1'}]",UNKNOWN,AUCTION
120211391375820669,Test_Ad_FerCreditGral,Test_Group_FerCreditEncinos,Test_All_Nov_Camp,2024-10-15,2024-11-13,1756.0,2734.0,1.556948,145.37,156.0,26.0,UNKNOWN,0.931859,82.784738,53.171178,"[{'action_type': 'onsite_conversion.total_messaging_connection', 'value': '2'}, {'action_type': 'post_engagement', 'value': '27'}, {'action_type': 'page_engagement', 'value': '27'}, {'action_type': 'onsite_conversion.messaging_conversation_started_7d', 'value': '1'}, {'action_type': 'post_reaction', 'value': '1'}, {'action_type': 'link_click', 'value': '26'}]",UNKNOWN,AUCTION
120211392468530669,Test_Ad_FerCredit_Domingo,Test_Group_FerCreditEncinos,Test_All_Nov_Camp,2024-10-15,2024-11-13,42.0,55.0,1.309524,4.02,2.0,null,UNKNOWN,2.01,95.714286,73.090909,null,UNKNOWN,AUCTION
120211362612890669,Test_Ad_BosqueDesertAlta,Test_Group_Ads_Generales,Test_All_Nov_Camp,2024-10-15,2024-11-13,4628.0,8159.0,1.762965,284.2,153.0,40.0,UNKNOWN,1.857516,61.408816,34.8327,"[{'action_type': 'onsite_conversion.total_messaging_connection', 'value': '1'}, {'action_type': 'post_engagement', 'value': '48'}, {'action_type': 'page_engagement', 'value': '49'}, {'action_type': 'onsite_conversion.messaging_user_depth_3_message_send', 'value': '1'}, {'action_type': 'post', 'value': '1'}, {'action_type': 'onsite_conversion.messaging_user_depth_2_message_send', 'value': '1'}, {'action_type': 'like', 'value': '1'}, {'action_type': 'onsite_conversion.messaging_conversation_started_7d', 'value': '1'}, {'action_type': 'post_reaction', 'value': '7'}, {'action_type': 'link_click', 'value': '40'}]",UNKNOWN,AUCTION
120211361743520669,Test_Ad_MemTraffic146Encinos,Test_Group_MemesTrafico,Test_All_Nov_Camp,2024-10-15,2024-11-13,1212.0,1732.0,1.429043,117.23,12.0,8.0,UNKNOWN,9.769167,96.724422,67.684758,"[{'action_type': 'post_engagement', 'value': '10'}, {'action_type': 'onsite_conversion.post_save', 'value': '1'}, {'action_type': 'page_engagement', 'value': '10'}, {'action_type': 'post_reaction', 'value': '1'}, {'action_type': 'link_click', 'value': '8'}]",UNKNOWN,AUCTION
120211362084550669,Test_Ad_MemTraffic106Alta,Test_Group_MemesTrafico,Test_All_Nov_Camp,2024-10-15,2024-11-13,1609.0,2585.0,1.606588,183.03,16.0,6.0,UNKNOWN,11.439375,113.753884,70.804642,"[{'action_type': 'post_engagement', 'value': '8'}, {'action_type': 'page_engagement', 'value': '8'}, {'action_type': 'post_reaction', 'value': '2'}, {'action_type': 'link_click', 'value': '6'}]",UNKNOWN,AUCTION
120211361894020669,Test_Ad_MemTrafficT80Altavista,Test_Group_MemesTrafico,Test_All_Nov_Camp,2024-10-15,2024-11-13,2746.0,4407.0,1.60488,306.86,54.0,25.0,UNKNOWN,5.682593,111.747997,69.630134,"[{

probando

In [0]:
#borrar archivos expecificos dentro de una carpeta
dbutils.fs.rm("/mnt/processed/Ads_GralTotal", True)


True